<a href="https://colab.research.google.com/github/lostgear84/exercicios_IA_python/blob/main/agenda_odonto_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import datetime
from datetime import timedelta
import random
import json
import os

# Simulação de integração com WhatsApp (substituir por Twilio ou WhatsApp Business API em produção)
class WhatsAppBot:
    def __init__(self):
        self.db_path = "dental_clinic.db"
        self.accepted_plans = ["unimed", "amil", "bradesco saúde"]  # Lista de convênios aceitos
        self.setup_database()
        self.conversation_state = {}
        self.feedback_data = self.load_feedback()

    def setup_database(self):
        # Configura o banco de dados SQLite
        conn = sqlite3.connect(self.db_path)
        c = conn.cursor()
        # Tabela de pacientes
        c.execute('''CREATE TABLE IF NOT EXISTS patients
                     (phone TEXT PRIMARY KEY, name TEXT)''')
        # Tabela de agendamentos
        c.execute('''CREATE TABLE IF NOT EXISTS appointments
                     (phone TEXT, date TEXT, time TEXT, type TEXT, plan TEXT)''')
        conn.commit()
        conn.close()

    def load_feedback(self):
        # Carrega dados de feedback para aprendizado
        if os.path.exists("feedback.json"):
            with open("feedback.json", "r") as f:
                return json.load(f)
        return {"successful": 0, "failed": 0, "responses": {}}

    def save_feedback(self):
        # Salva feedback para aprendizado
        with open("feedback.json", "w") as f:
            json.dump(self.feedback_data, f)

    def learn_from_feedback(self, phone, success):
        # Ajusta pontuação de aprendizado com base no sucesso da interação
        self.feedback_data["successful" if success else "failed"] += 1
        if phone not in self.feedback_data["responses"]:
            self.feedback_data["responses"][phone] = {"success": 0, "failed": 0}
        self.feedback_data["responses"][phone]["success" if success else "failed"] += 1
        self.save_feedback()

    def get_patient_name(self, phone):
        # Busca nome do paciente pelo número de telefone
        conn = sqlite3.connect(self.db_path)
        c = conn.cursor()
        c.execute("SELECT name FROM patients WHERE phone = ?", (phone,))
        result = c.fetchone()
        conn.close()
        return result[0] if result else None

    def check_availability(self, date, time):
        # Verifica disponibilidade na agenda
        conn = sqlite3.connect(self.db_path)
        c = conn.cursor()
        c.execute("SELECT * FROM appointments WHERE date = ? AND time = ?", (date, time))
        result = c.fetchone()
        conn.close()
        return result is None

    def suggest_alternative_slots(self, date):
        # Sugere 2 horários antes e 2 depois da data sugerida
        try:
            date_obj = datetime.datetime.strptime(date, "%d/%m")
            date_obj = date_obj.replace(year=datetime.datetime.now().year)
        except ValueError:
            return ["Data inválida"]
        alternatives = []
        for i in [-2, -1, 1, 2]:
            new_date = (date_obj + timedelta(days=i)).strftime("%d/%m")
            for time in ["09h00", "15h00"]:  # Horários de exemplo
                if self.check_availability(new_date, time):
                    alternatives.append(f"{new_date} às {time}")
        return alternatives[:4]

    def book_appointment(self, phone, date, time, appt_type, plan):
        # Registra o agendamento no banco de dados
        date_obj = datetime.datetime.strptime(date, "%d/%m")
        date_obj = date_obj.replace(year=datetime.datetime.now().year)
        formatted_date = date_obj.strftime("%d/%m")
        conn = sqlite3.connect(self.db_path)
        c = conn.cursor()
        c.execute("INSERT INTO appointments (phone, date, time, type, plan) VALUES (?, ?, ?, ?, ?)",
                  (phone, formatted_date, time, appt_type, plan))
        conn.commit()
        conn.close()

    def handle_message(self, phone, message):
        # Lógica do fluxo de conversa
        if phone not in self.conversation_state:
            self.conversation_state[phone] = {"step": "welcome"}

        state = self.conversation_state[phone]
        name = self.get_patient_name(phone)
        response = ""

        if state["step"] == "welcome":
            if name:
                response = f"Olá, {name}! Bem-vindo(a) ao agendamento do consultório odontológico. Sua consulta será particular ou convênio?"
            else:
                response = "Olá! Bem-vindo(a) ao agendamento do consultório odontológico. Sua consulta será particular ou convênio?"
            state["step"] = "plan"

        elif state["step"] == "plan":
            if message.lower() == "particular":
                state["plan"] = "particular"
                response = "Ótimo! É uma consulta ou retorno?"
                state["step"] = "type"
            elif message.lower() == "convênio":
                response = "Qual é o convênio?"
                state["step"] = "check_plan"
            else:
                response = "Por favor, informe se é particular ou convênio."

        elif state["step"] == "check_plan":
            if message.lower() in self.accepted_plans:
                state["plan"] = message.lower()
                response = "Ótimo! É uma consulta ou retorno?"
                state["step"] = "type"
            else:
                response = f"Infelizmente, não aceitamos o convênio {message}. Deseja prosseguir com um agendamento particular? (Sim/Não)"
                state["step"] = "plan_fallback"

        elif state["step"] == "plan_fallback":
            if message.lower() in ["sim", "s"]:
                state["plan"] = "particular"
                response = "Ótimo! É uma consulta ou retorno?"
                state["step"] = "type"
            else:
                response = "Entendido. Deseja falar com a secretária para outras opções?"
                state["step"] = "confirm"
                self.learn_from_feedback(phone, False)

        elif state["step"] == "type":
            if message.lower() in ["consulta", "retorno"]:
                state["type"] = message.lower()
                response = "Perfeito! Qual dia e horário você prefere? (Exemplo: 10/09 14h ou 10/09 14h30)"
                state["step"] = "datetime"
            else:
                response = "Por favor, informe se é consulta ou retorno."

        elif state["step"] == "datetime":
            try:
                date, time = message.split()
                # Normaliza formato do horário
                time = time.replace("h", "h").replace(":", "h")
                if len(time) == 2:
                    time += "h00"
                elif len(time) == 4:
                    time = time[:2] + "h" + time[2:]
                if not (time.endswith("h00") or time.endswith("h15") or time.endswith("h30") or time.endswith("h45")):
                    raise ValueError
                datetime.datetime.strptime(date, "%d/%m")
                if self.check_availability(date, time):
                    self.book_appointment(phone, date, time, state["type"], state["plan"])
                    response = f"Agendamento confirmado para {date} às {time}! É apenas isso?"
                    state["step"] = "confirm"
                    self.learn_from_feedback(phone, True)
                else:
                    alternatives = self.suggest_alternative_slots(date)
                    response = (f"Desculpe, o horário {date} às {time} não está disponível. "
                               f"Opções disponíveis: {', '.join(alternatives)}. Qual prefere?")
                    state["step"] = "retry_datetime"
            except ValueError:
                response = "Formato inválido. Por favor, envie a data e hora no formato 'DD/MM HHh' ou 'DD/MM HHhMM'."

        elif state["step"] == "retry_datetime":
            try:
                date, time = message.split()
                time = time.replace("h", "h").replace(":", "h")
                if len(time) == 2:
                    time += "h00"
                elif len(time) == 4:
                    time = time[:2] + "h" + time[2:]
                if not (time.endswith("h00") or time.endswith("h15") or time.endswith("h30") or time.endswith("h45")):
                    raise ValueError
                datetime.datetime.strptime(date, "%d/%m")
                if self.check_availability(date, time):
                    self.book_appointment(phone, date, time, state["type"], state["plan"])
                    response = f"Agendamento confirmado para {date} às {time}! É apenas isso?"
                    state["step"] = "confirm"
                    self.learn_from_feedback(phone, True)
                else:
                    response = "Horário indisponível. Tente outro horário ou entre em contato com a secretária."
                    state["step"] = "confirm"
                    self.learn_from_feedback(phone, False)
            except ValueError:
                response = "Formato inválido. Por favor, envie a data e hora no formato 'DD/MM HHh' ou 'DD/MM HHhMM'."

        elif state["step"] == "confirm":
            if message.lower() in ["sim", "s"]:
                response = "Obrigado por agendar conosco! Até breve!"
                self.conversation_state.pop(phone)
            else:
                response = "Encaminhando para a secretária. Aguarde um momento."
                self.conversation_state.pop(phone)
                self.learn_from_feedback(phone, False)

        return response

# Função principal para simular interação
def main():
    bot = WhatsAppBot()
    phone = "123456789"  # Exemplo de número de telefone
    print("Simulação de conversa via WhatsApp. Digite 'sair' para encerrar.")
    while True:
        message = input("Você: ")
        if message.lower() == "sair":
            break
        response = bot.handle_message(phone, message)
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()